In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [3]:
dataset = dataset.drop("User ID", axis=1)

In [4]:
dataset = pd.get_dummies(dataset, dtype=int, drop_first=True)
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [5]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [6]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [7]:
independent = dataset[['Age', 'EstimatedSalary', 'Gender_Male']]
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [8]:
dependent = dataset[['Purchased']]
dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(independent, dependent, random_state=0, test_size=0.30)

In [10]:
from sklearn.preprocessing import StandardScaler
scx = StandardScaler()
x_train = scx.fit_transform(x_train)
x_test = scx.transform(x_test)


In [11]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

param_grid = {
    "kernel": ["linear", "poly", "rbf", "sigmoid"],
    "gamma": ["scale", "auto"],
    "C" : [10, 100, 1000, 2000,3000]
}

grid = GridSearchCV(SVC(probability = True), param_grid, refit=True, verbose=3, n_jobs=-1, cv=5, scoring='f1_weighted')

grid.fit(x_train, y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


C:\Anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=5, estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [10, 100, 1000, 2000, 3000],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             scoring='f1_weighted', verbose=3)

In [12]:
y_pred = grid.predict(x_test)

In [13]:
from sklearn.metrics import f1_score
f1_macro = f1_score(y_test, y_pred, average="weighted")
print("The f1 macro value for the best parameter is {}:".format(grid.best_params_), f1_macro)

The f1 macro value for the best parameter is {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}: 0.9171245421245421


In [14]:
from sklearn.metrics import roc_auc_score
roc_score = roc_auc_score(y_test, grid.predict_proba(x_test)[:,1])
print("roc_auc score for the model is {}:".format(roc_score))

roc_auc score for the model is 0.9654214263661625:


In [15]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
cm = confusion_matrix(y_test, y_pred)
clf_report = classification_report(y_test, y_pred)

print(cm)
print(clf_report)

[[73  6]
 [ 4 37]]
              precision    recall  f1-score   support

           0       0.95      0.92      0.94        79
           1       0.86      0.90      0.88        41

    accuracy                           0.92       120
   macro avg       0.90      0.91      0.91       120
weighted avg       0.92      0.92      0.92       120



In [16]:
re = grid.cv_results_
table = pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.027742,0.007899,0.015370,0.000964,10,scale,linear,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}",0.835985,0.782971,0.644599,0.927778,0.890114,0.816289,0.098849,21
1,0.024007,0.003195,0.013907,0.003102,10,scale,poly,"{'C': 10, 'gamma': 'scale', 'kernel': 'poly'}",0.833024,0.799537,0.737557,0.928571,0.888158,0.837369,0.066798,12
2,0.019569,0.002866,0.013907,0.003266,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.892857,0.875644,0.841398,0.947015,0.946153,0.900613,0.041029,1
3,0.014972,0.003748,0.014543,0.001465,10,scale,sigmoid,"{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.769053,0.733523,0.610390,0.753871,0.759910,0.725349,0.058652,40
4,0.022507,0.002234,0.011494,0.003171,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.835985,0.782971,0.644599,0.927778,0.890114,0.816289,0.098849,21
5,0.022976,0.006462,0.012226,0.003607,10,auto,poly,"{'C': 10, 'gamma': 'auto', 'kernel': 'poly'}",0.833024,0.799537,0.737557,0.928571,0.888158,0.837369,0.066798,12
6,0.021550,0.001782,0.011379,0.005049,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",0.892857,0.875644,0.841398,0.947015,0.946153,0.900613,0.041029,1
7,0.014522,0.003664,0.011095,0.003956,10,auto,sigmoid,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.769053,0.733523,0.730543,0.753871,0.759910,0.749380,0.014996,32
8,0.066681,0.006865,0.014322,0.004313,100,scale,linear,"{'C': 100, 'gamma': 'scale', 'kernel': 'linear'}",0.835985,0.782971,0.644599,0.927778,0.890114,0.816289,0.098849,21
9,0.076104,0.021552,0.013815,0.003918,100,scale,poly,"{'C': 100, 'gamma': 'scale', 'kernel': 'poly'}",0.833024,0.799537,0.737557,0.928571,0.907658,0.841269,0.070136,11
